In [1]:
!pip install tensorboardX catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.7 MB/s eta 0:00:00


In [2]:
import os
import shutil

working_dir = '/kaggle/working'

# Iterate over all files and folders in the working directory
for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # remove file or symlink
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # remove directory
    except Exception as e:
        print(f"❌ Failed to delete {item_path}: {e}")

print("✅ /kaggle/working directory emptied successfully!")


✅ /kaggle/working directory emptied successfully!


In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import catboost as ctb
from scipy.spatial import distance
from itertools import groupby
from tensorboardX import SummaryWriter

In [3]:
#dataset class
class trackNetDataset(Dataset):
    def __init__(self, mode, input_width=1280 , input_height=720):
        self.path_dataset = '/kaggle/working/datasets/trackNet'  # Update path as needed
        assert mode in ['train', 'val'], 'incorrect mode'
        self.data = pd.read_csv(os.path.join(self.path_dataset, 'labels_{}.csv'.format(mode)))
        print('mode = {}, samples = {}'.format(mode, self.data.shape[0]))         
        self.height = input_height
        self.width = input_width
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        path, path_prev, path_preprev, path_gt, x, y, status, vis = self.data.loc[idx, :]
        
        path = os.path.join(self.path_dataset, path)
        path_prev = os.path.join(self.path_dataset, path_prev)
        path_preprev = os.path.join(self.path_dataset, path_preprev)
        path_gt = os.path.join(self.path_dataset, path_gt)
        if math.isnan(x):
            x = -1
            y = -1
        
        inputs = self.get_input(path, path_prev, path_preprev)
        outputs = self.get_output(path_gt)
        
        return inputs, outputs, x, y, vis
    
    def get_output(self, path_gt):
        img = cv2.imread(path_gt)
        img = cv2.resize(img, (self.width, self.height))
        img = img[:, :, 0]
        img = np.reshape(img, (self.width * self.height))
        return img
        
    def get_input(self, path, path_prev, path_preprev):
        img = cv2.imread(path)
        img = cv2.resize(img, (self.width, self.height))

        img_prev = cv2.imread(path_prev)
        img_prev = cv2.resize(img_prev, (self.width, self.height))
        
        img_preprev = cv2.imread(path_preprev)
        img_preprev = cv2.resize(img_preprev, (self.width, self.height))
        
        imgs = np.concatenate((img, img_prev, img_preprev), axis=2)
        imgs = imgs.astype(np.float32)/255.0

        imgs = np.rollaxis(imgs, 2, 0)
        return imgs

In [4]:
#Model defination

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, pad=1, stride=1, bias=True):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=pad, bias=bias),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.block(x)

class BallTrackerNet(nn.Module):
    def __init__(self, out_channels=256):
        super().__init__()
        self.out_channels = out_channels

        self.conv1 = ConvBlock(in_channels=9, out_channels=64)
        self.conv2 = ConvBlock(in_channels=64, out_channels=64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = ConvBlock(in_channels=64, out_channels=128)
        self.conv4 = ConvBlock(in_channels=128, out_channels=128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = ConvBlock(in_channels=128, out_channels=256)
        self.conv6 = ConvBlock(in_channels=256, out_channels=256)
        self.conv7 = ConvBlock(in_channels=256, out_channels=256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv8 = ConvBlock(in_channels=256, out_channels=512)
        self.conv9 = ConvBlock(in_channels=512, out_channels=512)
        self.conv10 = ConvBlock(in_channels=512, out_channels=512)
        self.ups1 = nn.Upsample(scale_factor=2)
        self.conv11 = ConvBlock(in_channels=512, out_channels=256)
        self.conv12 = ConvBlock(in_channels=256, out_channels=256)
        self.conv13 = ConvBlock(in_channels=256, out_channels=256)
        self.ups2 = nn.Upsample(scale_factor=2)
        self.conv14 = ConvBlock(in_channels=256, out_channels=128)
        self.conv15 = ConvBlock(in_channels=128, out_channels=128)
        self.ups3 = nn.Upsample(scale_factor=2)
        self.conv16 = ConvBlock(in_channels=128, out_channels=64)
        self.conv17 = ConvBlock(in_channels=64, out_channels=64)
        self.conv18 = ConvBlock(in_channels=64, out_channels=self.out_channels)

        self.softmax = nn.Softmax(dim=1)
        self._init_weights()
                  
    def forward(self, x, testing=False): 
        batch_size = x.size(0)
        x = self.conv1(x)
        x = self.conv2(x)    
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.ups1(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.ups2(x)
        x = self.conv14(x)
        x = self.conv15(x)
        x = self.ups3(x)
        x = self.conv16(x)
        x = self.conv17(x)
        x = self.conv18(x)
        out = x.reshape(batch_size, self.out_channels, -1)
        if testing:
            out = self.softmax(out)
        return out                       
    
    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.uniform_(module.weight, -0.05, 0.05)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.BatchNorm2d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)

In [5]:
#Utility functions

def postprocess(feature_map, scale=2):
    feature_map *= 255
    feature_map = feature_map.reshape((360, 640))
    feature_map = feature_map.astype(np.uint8)
    ret, heatmap = cv2.threshold(feature_map, 127, 255, cv2.THRESH_BINARY)
    circles = cv2.HoughCircles(heatmap, cv2.HOUGH_GRADIENT, dp=1, minDist=1, param1=50, param2=2, minRadius=2,
                               maxRadius=7)
    x,y = None, None
    if circles is not None:
        if len(circles) == 1:
            x = circles[0][0][0]*scale
            y = circles[0][0][1]*scale
    return x, y

def train(model, train_loader, optimizer, device, epoch, max_iters=200):
    start_time = time.time()
    losses = []
    criterion = nn.CrossEntropyLoss()
    for iter_id, batch in enumerate(train_loader):
        optimizer.zero_grad()
        model.train()
        out = model(batch[0].float().to(device))
        gt = torch.tensor(batch[1], dtype=torch.long, device=device)
        loss = criterion(out, gt)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        end_time = time.time()
        duration = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))
        print('train | epoch = {}, iter = [{}|{}], loss = {}, time = {}'.format(epoch, iter_id, max_iters,
                                                                                round(loss.item(), 6), duration))
        losses.append(loss.item())
        
        if iter_id > max_iters - 1:
            break
        
    return np.mean(losses)

def validate(model, val_loader, device, epoch, min_dist=5):
    losses = []
    tp = [0, 0, 0, 0]
    fp = [0, 0, 0, 0]
    tn = [0, 0, 0, 0]
    fn = [0, 0, 0, 0]
    criterion = nn.CrossEntropyLoss()
    model.eval()
    for iter_id, batch in enumerate(val_loader):
        with torch.no_grad():
            out = model(batch[0].float().to(device))
            gt = torch.tensor(batch[1], dtype=torch.long, device=device)
            loss = criterion(out, gt)
            losses.append(loss.item())
            # metrics
            output = out.argmax(dim=1).detach().cpu().numpy()
            for i in range(len(output)):
                x_pred, y_pred = postprocess(output[i])
                x_gt = batch[2][i]
                y_gt = batch[3][i]
                vis = batch[4][i]
                if x_pred:
                    if vis != 0:
                        dst = distance.euclidean((x_pred, y_pred), (x_gt, y_gt))
                        if dst < min_dist:
                            tp[vis] += 1
                        else:
                            fp[vis] += 1
                    else:        
                        fp[vis] += 1
                if not x_pred:
                    if vis != 0:
                        fn[vis] += 1
                    else:
                        tn[vis] += 1
            print('val | epoch = {}, iter = [{}|{}], loss = {}, tp = {}, tn = {}, fp = {}, fn = {} '.format(epoch,
                                                                                                            iter_id,
                                                                                                            len(val_loader),
                                                                                                            round(np.mean(losses), 6),
                                                                                                            sum(tp),
                                                                                                            sum(tn),
                                                                                                            sum(fp),
                                                                                                            sum(fn)))
    eps = 1e-15
    precision = sum(tp) / (sum(tp) + sum(fp) + eps)
    vc1 = tp[1] + fp[1] + tn[1] + fn[1]
    vc2 = tp[2] + fp[2] + tn[2] + fn[2]
    vc3 = tp[3] + fp[3] + tn[3] + fn[3]
    recall = sum(tp) / (vc1 + vc2 + vc3 + eps)
    f1 = 2 * precision * recall / (precision + recall + eps)
    print('precision = {}'.format(precision))
    print('recall = {}'.format(recall))
    print('f1 = {}'.format(f1))

    return np.mean(losses), precision, recall, f1

In [9]:
import shutil
#shutil.rmtree('/kaggle/working/exps')
import os

file_path = '/kaggle/working/output_video.avi'
if os.path.exists(file_path):
    os.remove(file_path)
    print("File removed successfully!")
else:
    print("File does not exist.")



File removed successfully!


# **# Test on video for ball detection**

In [10]:
import torch
import cv2
from tqdm import tqdm
import numpy as np
from itertools import groupby
from scipy.spatial import distance
import os

# ========== Parameters ==========
BATCH_SIZE = 2
MODEL_PATH = "/kaggle/input/tennis_best_model/pytorch/default/1/model_best.pth"
VIDEO_PATH = "/kaggle/input/test-tennis-dataset/test_dataset/video8.mp4"
VIDEO_OUT_PATH = "/kaggle/working/output_video.avi"
USE_EXTRAPOLATION = True

# ========== Utils ==========
def read_video(path_video):
    cap = cv2.VideoCapture(path_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames, fps

def infer_model(frames, model):
    height = 360
    width = 640
    dists = [-1]*2
    ball_track = [(None,None)]*2
    for num in tqdm(range(2, len(frames))):
        img = cv2.resize(frames[num], (width, height))
        img_prev = cv2.resize(frames[num-1], (width, height))
        img_preprev = cv2.resize(frames[num-2], (width, height))
        imgs = np.concatenate((img, img_prev, img_preprev), axis=2)
        imgs = imgs.astype(np.float32)/255.0
        imgs = np.rollaxis(imgs, 2, 0)
        inp = np.expand_dims(imgs, axis=0)

        out = model(torch.from_numpy(inp).float().to(device))
        output = out.argmax(dim=1).detach().cpu().numpy()
        x_pred, y_pred = postprocess(output)
        ball_track.append((x_pred, y_pred))

        if ball_track[-1][0] and ball_track[-2][0]:
            dist = distance.euclidean(ball_track[-1], ball_track[-2])
        else:
            dist = -1
        dists.append(dist)
    return ball_track, dists

def remove_outliers(ball_track, dists, max_dist = 100):
    outliers = list(np.where(np.array(dists) > max_dist)[0])
    for i in outliers:
        if (dists[i+1] > max_dist) or (dists[i+1] == -1):
            ball_track[i] = (None, None)
            outliers.remove(i)
        elif dists[i-1] == -1:
            ball_track[i-1] = (None, None)
    return ball_track  

def split_track(ball_track, max_gap=4, max_dist_gap=80, min_track=5):
    list_det = [0 if x[0] else 1 for x in ball_track]
    groups = [(k, sum(1 for _ in g)) for k, g in groupby(list_det)]

    cursor = 0
    min_value = 0
    result = []
    for i, (k, l) in enumerate(groups):
        if (k == 1) and (i > 0) and (i < len(groups) - 1):
            dist = distance.euclidean(ball_track[cursor-1], ball_track[cursor+l])
            if (l >= max_gap) or (dist/l > max_dist_gap):
                if cursor - min_value > min_track:
                    result.append([min_value, cursor])
                    min_value = cursor + l - 1
        cursor += l
    if len(list_det) - min_value > min_track:
        result.append([min_value, len(list_det)])
    return result

def interpolation(coords):
    def nan_helper(y):
        return np.isnan(y), lambda z: z.nonzero()[0]

    x = np.array([x[0] if x[0] is not None else np.nan for x in coords])
    y = np.array([x[1] if x[1] is not None else np.nan for x in coords])

    nons, yy = nan_helper(x)
    x[nons] = np.interp(yy(nons), yy(~nons), x[~nons])
    nans, xx = nan_helper(y)
    y[nans] = np.interp(xx(nans), xx(~nans), y[~nans])

    track = list(zip(x, y))
    return track

def write_track(frames, ball_track, path_output_video, fps, trace=7):
    height, width = frames[0].shape[:2]
    out = cv2.VideoWriter(path_output_video, cv2.VideoWriter_fourcc(*'DIVX'),
                          fps, (width, height))
    for num in range(len(frames)):
        frame = frames[num]
        for i in range(trace):
            if (num - i > 0):
                if ball_track[num - i][0]:
                    x = int(ball_track[num - i][0])
                    y = int(ball_track[num - i][1])
                    frame = cv2.circle(frame, (x, y), radius=0, color=(0, 0, 255), thickness=10 - i)
                else:
                    break
        out.write(frame)
    out.release()

# ========== Run ==========
model = BallTrackerNet()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()

frames, fps = read_video(VIDEO_PATH)
ball_track, dists = infer_model(frames, model)
ball_track = remove_outliers(ball_track, dists)

if USE_EXTRAPOLATION:
    subtracks = split_track(ball_track)
    for r in subtracks:
        ball_subtrack = ball_track[r[0]:r[1]]
        ball_subtrack = interpolation(ball_subtrack)
        ball_track[r[0]:r[1]] = ball_subtrack

write_track(frames, ball_track, VIDEO_OUT_PATH, fps)
print(f"Output video saved to {VIDEO_OUT_PATH}")


/tmp/ipykernel_31/141128064.py:120: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
100%|██████████| 481/481

Output video saved to /kaggle/working/output_video.avi


# ** Test for key-point detection**# # # 

In [13]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [14]:
import cv2
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

In [15]:
COURT_MODEL_PATH = "/kaggle/input/keypoint_model/pytorch/default/1/keypoints_model.pth"
TRACKNET_VIDEO_PATH = "/kaggle/working/output_video.avi"
COMBINED_VIDEO_OUT_PATH = "/kaggle/working/output_video_with_ball_and_keypoints.avi"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


In [16]:
def read_video(path_video):
    cap = cv2.VideoCapture(path_video)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video file.")
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames, fps

def write_combined_video(frames, keypoints_list, path_output_video, fps):
    height, width = frames[0].shape[:2]
    out = cv2.VideoWriter(path_output_video, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    for num in range(len(frames)):
        frame = frames[num]
        # Draw court keypoints
        if num < len(keypoints_list):
            for i, kp in enumerate(keypoints_list[num]):
                x, y = kp
                x, y = int(x), int(y)
                frame = cv2.circle(frame, (x, y), radius=5, color=(0, 255, 0), thickness=-1)
                frame = cv2.putText(frame, f'KP{i}', (x + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        out.write(frame)
    out.release()

def process_video_with_court_keypoint_detection(video_path, court_model_path):
    # Define ResNet50 model for 14 keypoints
    model = models.resnet50(weights=None)  # No pre-trained weights; we'll load custom weights
    model.fc = torch.nn.Linear(model.fc.in_features, 14 * 2)  # 14 keypoints (x, y)
    
    # Load model weights
    try:
        state_dict = torch.load(court_model_path, map_location=DEVICE, weights_only=True)
        model.load_state_dict(state_dict)
    except Exception as e:
        raise ValueError(
            f"Failed to load weights from {court_model_path}. Ensure the file contains valid weights "
            f"for a ResNet50 model modified for 14 keypoints (28 outputs). Error: {str(e)}"
        )
    
    model.to(DEVICE)
    model.eval()
    
    # Define image transformations
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Read TrackNet output video
    frames, fps = read_video(video_path)
    
    # Process frames for court keypoint detection
    keypoints_list = []
    for frame in tqdm(frames, desc="Processing frames for court keypoint detection"):
        # Preprocess frame
        original_h, original_w = frame.shape[:2]
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_tensor = transform(img_rgb).unsqueeze(0).to(DEVICE)
        
        # Run inference
        with torch.no_grad():
            outputs = model(img_tensor)
        
        # Extract and scale keypoints
        keypoints = outputs.squeeze().cpu().numpy()
        keypoints = keypoints.reshape(14, 2)  # 14 keypoints, each with (x, y)
        # Scale keypoints to original frame size
        keypoints[:, 0] *= original_w / 224.0  # x-coordinates
        keypoints[:, 1] *= original_h / 224.0  # y-coordinates
        
        keypoints_list.append(keypoints)
    
    # Write combined video
    write_combined_video(frames, keypoints_list, COMBINED_VIDEO_OUT_PATH, fps)
    print(f"Combined video saved to {COMBINED_VIDEO_OUT_PATH}")

# Run court keypoint detection on TrackNet output video
process_video_with_court_keypoint_detection(TRACKNET_VIDEO_PATH, COURT_MODEL_PATH)

Processing frames for court keypoint detection: 100%|██████████| 483/483 [00:07<00:00, 67.69it/s]


Combined video saved to /kaggle/working/output_video_with_ball_and_keypoints.avi


In [17]:
# Cell 20: Verify combined output video
cap = cv2.VideoCapture(COMBINED_VIDEO_OUT_PATH)
if not cap.isOpened():
    print("Error: Combined video corrupted!")
else:
    print(f"Combined video ready! Frames: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}")
cap.release()

Combined video ready! Frames: 483


# **# Test For Player detection : **# 

In [19]:
!pip install ultralytics

In [20]:
import cv2
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [22]:
# Parameters
PLAYER_MODEL_PATH = "/kaggle/input/player_detection_model/pytorch/default/1/best.pt"
TRACKNET_VIDEO_PATH = "/kaggle/working/output_video_with_ball_and_keypoints.avi"
COMBINED_VIDEO_OUT_PATH = "/kaggle/working/output_video_with_ball_and_players.avi"

In [23]:
def read_video(path_video):
    cap = cv2.VideoCapture(path_video)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video file.")
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames, fps

def write_combined_video(frames, player_bboxes, path_output_video, fps):
    height, width = frames[0].shape[:2]
    out = cv2.VideoWriter(path_output_video, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    for num in range(len(frames)):
        frame = frames[num]
        # Draw player bounding boxes
        if num < len(player_bboxes) and player_bboxes[num]:
            for track_id, bbox in player_bboxes[num].items():
                x1, y1, x2, y2 = map(int, bbox)
                # Ensure coordinates are within frame bounds
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(width, x2), min(height, y2)
                if x2 > x1 and y2 > y1:  # Valid bounding box
                    label = f'Player ID: {track_id}'
                    frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue color
                    frame = cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        out.write(frame)
    out.release()

def process_video_with_player_detection(video_path, player_model_path):
    # Load YOLOv8 model
    try:
        model = YOLO(player_model_path)
        model.to(DEVICE)
    except Exception as e:
        raise ValueError(f"Failed to load YOLOv8 model from {player_model_path}. Error: {str(e)}")
    
    # Print model class names for debugging
    print(f"Model class names: {model.names}")
    
    # Read TrackNet output video
    frames, fps = read_video(video_path)
    
    # Process frames for player detection
    player_bboxes = []
    no_detection_frames = 0
    for frame in tqdm(frames, desc="Processing frames for player detection"):
        # Detect players (without tracking to simplify)
        results = model.predict(frame)[0]
        id_name_dict = results.names
        player_dict = {}
        
        for box in results.boxes:
            object_cls_id = int(box.cls.item())
            object_cls_name = id_name_dict[object_cls_id]
            # Allow "person" or "player" classes
            if object_cls_name in ["person", "player"]:
                x1, y1, x2, y2 = box.xyxy.tolist()[0]
                # Use index as pseudo-track ID if tracking is disabled
                track_id = len(player_dict) + 1
                player_dict[track_id] = [x1, y1, x2, y2]
        
        if not player_dict:
            no_detection_frames += 1
            print(f"Frame {len(player_bboxes)}: No players detected")
        
        player_bboxes.append(player_dict)
    
    # Log detection summary
    print(f"Total frames: {len(frames)}")
    print(f"Frames with no player detections: {no_detection_frames}")
    
    # Write combined video
    write_combined_video(frames, player_bboxes, COMBINED_VIDEO_OUT_PATH, fps)
    print(f"Combined video saved to {COMBINED_VIDEO_OUT_PATH}")

# Run player detection on TrackNet output video
process_video_with_player_detection(TRACKNET_VIDEO_PATH, PLAYER_MODEL_PATH)

Model class names: {0: 'ball', 1: 'player'}


Processing frames for player detection:   0%|          | 0/483 [00:00<?, ?it/s]


0: 384x640 2 players, 62.4ms
Speed: 2.8ms preprocess, 62.4ms inference, 248.1ms postprocess per image at shape (1, 3, 384, 640)


Processing frames for player detection:   0%|          | 1/483 [00:00<05:41,  1.41it/s]


0: 384x640 2 players, 8.0ms
Speed: 3.3ms preprocess, 8.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 7.8ms
Speed: 1.5ms preprocess, 7.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


...

Processing frames for player detection: 100%|█████████▉| 482/483 [00:06<00:00, 78.81it/s]


0: 384x640 3 players, 7.1ms
Speed: 1.6ms preprocess, 7.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing frames for player detection: 100%|██████████| 483/483 [00:07<00:00, 68.96it/s]

Total frames: 483
Frames with no player detections: 0


Combined video saved to /kaggle/working/output_video_with_ball_and_players.avi


In [25]:
# Cell 20: Verify combined output video
cap = cv2.VideoCapture(COMBINED_VIDEO_OUT_PATH)
if not cap.isOpened():
    print("Error: Combined video corrupted!")
else:
    print(f"Combined video ready! Frames: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}")
cap.release()

Combined video ready! Frames: 483


# **# Analysis**

In [28]:
import torch
import torchvision.transforms as transforms
import cv2
from torchvision import models
import numpy as np

class CourtLineDetector:
    def __init__(self, model_path):
        self.model = models.resnet50(pretrained=True)
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, 14*2) 
        self.model.load_state_dict(torch.load(model_path, map_location='cpu'))
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def predict(self, image):

    
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = self.transform(image_rgb).unsqueeze(0)
        with torch.no_grad():
            outputs = self.model(image_tensor)
        keypoints = outputs.squeeze().cpu().numpy()
        original_h, original_w = image.shape[:2]
        keypoints[::2] *= original_w / 224.0
        keypoints[1::2] *= original_h / 224.0

        return keypoints

    def draw_keypoints(self, image, keypoints):
        # Plot keypoints on the image
        for i in range(0, len(keypoints), 2):
            x = int(keypoints[i])
            y = int(keypoints[i+1])
            cv2.putText(image, str(i//2), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
        return image
    
    def draw_keypoints_on_video(self, video_frames, keypoints):
        output_video_frames = []
        for frame in video_frames:
            frame = self.draw_keypoints(frame, keypoints)
            output_video_frames.append(frame)
        return output_video_frames

In [36]:
class constants:
    SINGLE_LINE_WIDTH = 8.23
    DOUBLE_LINE_WIDTH = 10.97
    HALF_COURT_LINE_HEIGHT = 11.88
    SERVICE_LINE_WIDTH = 6.4
    DOUBLE_ALLY_DIFFERENCE = 1.37
    NO_MANS_LAND_HEIGHT = 5.48
    
    PLAYER_1_HEIGHT_METERS = 1.88
    PLAYER_2_HEIGHT_METERS = 1.91

In [32]:

def convert_pixel_distance_to_meters(pixel_distance, refrence_height_in_meters, refrence_height_in_pixels):
    return (pixel_distance * refrence_height_in_meters) / refrence_height_in_pixels

def convert_meters_to_pixel_distance(meters, refrence_height_in_meters, refrence_height_in_pixels):
    return (meters * refrence_height_in_pixels) / refrence_height_in_meters
def get_center_of_bbox(bbox):
    x1, y1, x2, y2 = bbox
    center_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    return (center_x, center_y)
def measure_distance(p1,p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
def get_foot_position(bbox):
    x1, y1, x2, y2 = bbox
    return (int((x1 + x2) / 2), y2)

def get_closest_keypoint_index(point, keypoints, keypoint_indices):
   closest_distance = float('inf')
   key_point_ind = keypoint_indices[0]
   for keypoint_indix in keypoint_indices:
       keypoint = keypoints[keypoint_indix*2], keypoints[keypoint_indix*2+1]
       distance = abs(point[1]-keypoint[1])

       if distance<closest_distance:
           closest_distance = distance
           key_point_ind = keypoint_indix
    
   return key_point_ind

def get_height_of_bbox(bbox):
    return bbox[3]-bbox[1]

def measure_xy_distance(p1,p2):
    return abs(p1[0]-p2[0]), abs(p1[1]-p2[1])

def get_center_of_bbox(bbox):
    return (int((bbox[0]+bbox[2])/2),int((bbox[1]+bbox[3])/2))

In [57]:
import cv2
import numpy as np
import sys
sys.path.append('../')

class MiniCourt():
    def __init__(self, frame):
        self.drawing_rectangle_width = 250
        self.drawing_rectangle_height = 500
        self.buffer = 50
        self.padding_court = 20

        self.set_canvas_background_box_position(frame)
        self.set_mini_court_position()
        self.set_court_drawing_key_points()
        self.set_court_lines()

    def convert_meters_to_pixels(self, meters):
        return convert_meters_to_pixel_distance(meters,
                                               constants.DOUBLE_LINE_WIDTH,
                                               self.court_drawing_width)

    def set_court_drawing_key_points(self):
        drawing_key_points = [0]*28

        # point 0 
        drawing_key_points[0], drawing_key_points[1] = int(self.court_start_x), int(self.court_start_y)
        # point 1
        drawing_key_points[2], drawing_key_points[3] = int(self.court_end_x), int(self.court_start_y)
        # point 2
        drawing_key_points[4] = int(self.court_start_x)
        drawing_key_points[5] = self.court_start_y + self.convert_meters_to_pixels(constants.HALF_COURT_LINE_HEIGHT*2)
        # point 3
        drawing_key_points[6] = drawing_key_points[0] + self.court_drawing_width
        drawing_key_points[7] = drawing_key_points[5] 
        # point 4
        drawing_key_points[8] = drawing_key_points[0] + self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[9] = drawing_key_points[1] 
        # point 5
        drawing_key_points[10] = drawing_key_points[4] + self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[11] = drawing_key_points[5] 
        # point 6
        drawing_key_points[12] = drawing_key_points[2] - self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[13] = drawing_key_points[3] 
        # point 7
        drawing_key_points[14] = drawing_key_points[6] - self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[15] = drawing_key_points[7] 
        # point 8
        drawing_key_points[16] = drawing_key_points[8] 
        drawing_key_points[17] = drawing_key_points[9] + self.convert_meters_to_pixels(constants.NO_MANS_LAND_HEIGHT)
        # point 9
        drawing_key_points[18] = drawing_key_points[16] + self.convert_meters_to_pixels(constants.SINGLE_LINE_WIDTH)
        drawing_key_points[19] = drawing_key_points[17] 
        # point 10
        drawing_key_points[20] = drawing_key_points[10] 
        drawing_key_points[21] = drawing_key_points[11] - self.convert_meters_to_pixels(constants.NO_MANS_LAND_HEIGHT)
        # point 11
        drawing_key_points[22] = drawing_key_points[20] + self.convert_meters_to_pixels(constants.SINGLE_LINE_WIDTH)
        drawing_key_points[23] = drawing_key_points[21] 
        # point 12
        drawing_key_points[24] = int((drawing_key_points[16] + drawing_key_points[18])/2)
        drawing_key_points[25] = drawing_key_points[17] 
        # point 13
        drawing_key_points[26] = int((drawing_key_points[20] + drawing_key_points[22])/2)
        drawing_key_points[27] = drawing_key_points[21] 

        self.drawing_key_points = drawing_key_points

    def set_court_lines(self):
        self.lines = [
            (0, 2),
            (4, 5),
            (6, 7),
            (1, 3),
            (0, 1),
            (8, 9),
            (10, 11),
            (10, 11),
            (2, 3)
        ]

    def set_mini_court_position(self):
        self.court_start_x = self.start_x + self.padding_court
        self.court_start_y = self.start_y + self.padding_court
        self.court_end_x = self.end_x - self.padding_court
        self.court_end_y = self.end_y - self.padding_court
        self.court_drawing_width = self.court_end_x - self.court_start_x

    def set_canvas_background_box_position(self, frame):
        frame = frame.copy()
        self.end_x = frame.shape[1] - self.buffer
        self.end_y = self.buffer + self.drawing_rectangle_height
        self.start_x = self.end_x - self.drawing_rectangle_width
        self.start_y = self.end_y - self.drawing_rectangle_height

    def draw_court(self, frame):
        for i in range(0, len(self.drawing_key_points), 2):
            x = int(self.drawing_key_points[i])
            y = int(self.drawing_key_points[i+1])
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)

        # Draw Lines
        for line in self.lines:
            start_point = (int(self.drawing_key_points[line[0]*2]), int(self.drawing_key_points[line[0]*2+1]))
            end_point = (int(self.drawing_key_points[line[1]*2]), int(self.drawing_key_points[line[1]*2+1]))
            cv2.line(frame, start_point, end_point, (0, 0, 0), 2)

        # Draw net
        net_start_point = (self.drawing_key_points[0], int((self.drawing_key_points[1] + self.drawing_key_points[5])/2))
        net_end_point = (self.drawing_key_points[2], int((self.drawing_key_points[1] + self.drawing_key_points[5])/2))
        cv2.line(frame, net_start_point, net_end_point, (255, 0, 0), 2)

        return frame

    def draw_background_rectangle(self, frame):
        shapes = np.zeros_like(frame, np.uint8)
        cv2.rectangle(shapes, (self.start_x, self.start_y), (self.end_x, self.end_y), (255, 255, 255), cv2.FILLED)
        out = frame.copy()
        alpha = 0.5
        mask = shapes.astype(bool)
        out[mask] = cv2.addWeighted(frame, alpha, shapes, 1 - alpha, 0)[mask]
        return out

    def draw_mini_court(self, frames):
        output_frames = []
        for frame in frames:
            frame = self.draw_background_rectangle(frame)
            frame = self.draw_court(frame)
            output_frames.append(frame)
        return output_frames

    def get_start_point_of_mini_court(self):
        return (self.court_start_x, self.court_start_y)

    def get_width_of_mini_court(self):
        return self.court_drawing_width

    def get_court_drawing_keypoints(self):
        return self.drawing_key_points

    def get_mini_court_coordinates(self,
                                   object_position,
                                   closest_key_point, 
                                   closest_key_point_index, 
                                   reference_height_in_pixels,
                                   reference_height_in_meters):
        distance_from_keypoint_x_pixels, distance_from_keypoint_y_pixels = measure_xy_distance(object_position, closest_key_point)
        distance_from_keypoint_x_meters = convert_pixel_distance_to_meters(distance_from_keypoint_x_pixels,
                                                                         reference_height_in_meters,
                                                                         reference_height_in_pixels)
        distance_from_keypoint_y_meters = convert_pixel_distance_to_meters(distance_from_keypoint_y_pixels,
                                                                         reference_height_in_meters,
                                                                         reference_height_in_pixels)
        mini_court_x_distance_pixels = self.convert_meters_to_pixels(distance_from_keypoint_x_meters)
        mini_court_y_distance_pixels = self.convert_meters_to_pixels(distance_from_keypoint_y_meters)
        closest_mini_court_keypoint = (self.drawing_key_points[closest_key_point_index*2],
                                      self.drawing_key_points[closest_key_point_index*2+1])
        mini_court_position = (closest_mini_court_keypoint[0] + mini_court_x_distance_pixels,
                               closest_mini_court_keypoint[1] + mini_court_y_distance_pixels)
        return mini_court_position

    def convert_bounding_boxes_to_mini_court_coordinates(self, player_boxes, ball_boxes, original_court_key_points):
        output_player_boxes = []
        output_ball_boxes = []

        # Default reference height for ball (since no player data is available)
        default_reference_height_meters = constants.PLAYER_1_HEIGHT_METERS  # Arbitrary, can adjust
        default_reference_height_pixels = 100  # Approximate, can adjust based on video

        for frame_num, ball_bbox in enumerate(ball_boxes):
            output_player_bboxes_dict = {}
            ball_box = ball_bbox.get(1, [None, None, None, None])
            if ball_box[0] is None:  # Skip if no ball detection
                output_ball_boxes.append({1: [None, None]})
                output_player_boxes.append(output_player_bboxes_dict)
                continue

            ball_position = get_center_of_bbox(ball_box)
            # Use court keypoints to find closest reference point
            closest_key_point_index = get_closest_keypoint_index(ball_position, original_court_key_points, [0, 2, 12, 13])
            closest_key_point = (original_court_key_points[closest_key_point_index*2], 
                                 original_court_key_points[closest_key_point_index*2+1])

            # Convert ball position to mini-court coordinates
            mini_court_ball_position = self.get_mini_court_coordinates(
                ball_position,
                closest_key_point,
                closest_key_point_index,
                default_reference_height_pixels,
                default_reference_height_meters
            )
            output_ball_boxes.append({1: mini_court_ball_position})
            output_player_boxes.append(output_player_bboxes_dict)

        return output_player_boxes, output_ball_boxes

    def draw_points_on_mini_court(self, frames, positions, color=(0, 255, 0)):
        for frame_num, frame in enumerate(frames):
            for _, position in positions[frame_num].items():
                if position[0] is None:
                    continue
                x, y = position
                x = int(x)
                y = int(y)
                cv2.circle(frame, (x, y), 5, color, -1)
        return frames

In [55]:
class BallTracker:
    def __init__(self, model_path):
        self.model = BallTrackerNet()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)
        self.model.eval()

    def interpolate_ball_positions(self, ball_positions):
        # Extract coordinates, using None for missing detections
        ball_coords = [x.get(1, [None, None, None, None])[0:4] for x in ball_positions]
        # Convert to pandas DataFrame
        df_ball_positions = pd.DataFrame(ball_coords, columns=['x1', 'y1', 'x2', 'y2'])
        # Interpolate missing values
        df_ball_positions = df_ball_positions.interpolate()
        df_ball_positions = df_ball_positions.bfill()
        # Convert back to list of dictionaries
        ball_positions = [{1: [x1, y1, x2, y2]} for x1, y1, x2, y2 in df_ball_positions.to_numpy().tolist()]
        return ball_positions

    def get_ball_shot_frames(self, ball_positions):
        # Extract coordinates
        ball_coords = [x.get(1, [None, None, None, None])[0:4] for x in ball_positions]
        df_ball_positions = pd.DataFrame(ball_coords, columns=['x1', 'y1', 'x2', 'y2'])
        df_ball_positions['ball_hit'] = 0
        # Compute rolling mean of y-coordinate
        df_ball_positions['mid_y'] = (df_ball_positions['y1'] + df_ball_positions['y2']) / 2
        df_ball_positions['y_rolling_mean'] = df_ball_positions['mid_y'].rolling(window=5, min_periods=1, center=False).mean()
        df_ball_positions['delta_y'] = df_ball_positions['y_rolling_mean'].diff()
        minimum_change_frames_for_hit = 25
        for i in range(1, len(df_ball_positions) - int(minimum_change_frames_for_hit * 1.2)):
            negative_position_change = df_ball_positions['delta_y'].iloc[i] > 0 and df_ball_positions['delta_y'].iloc[i + 1] < 0
            positive_position_change = df_ball_positions['delta_y'].iloc[i] < 0 and df_ball_positions['delta_y'].iloc[i + 1] > 0
            if negative_position_change or positive_position_change:
                change_count = 0
                for change_frame in range(i + 1, i + int(minimum_change_frames_for_hit * 1.2) + 1):
                    negative_position_change_following_frame = df_ball_positions['delta_y'].iloc[i] > 0 and df_ball_positions['delta_y'].iloc[change_frame] < 0
                    positive_position_change_following_frame = df_ball_positions['delta_y'].iloc[i] < 0 and df_ball_positions['delta_y'].iloc[change_frame] > 0
                    if negative_position_change and negative_position_change_following_frame:
                        change_count += 1
                    elif positive_position_change and positive_position_change_following_frame:
                        change_count += 1
                if change_count > minimum_change_frames_for_hit - 1:
                    df_ball_positions.loc[i, 'ball_hit'] = 1
        frame_nums_with_ball_hits = df_ball_positions[df_ball_positions['ball_hit'] == 1].index.tolist()
        return frame_nums_with_ball_hits

    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        if read_from_stub and stub_path is not None:
            with open(stub_path, 'rb') as f:
                ball_detections = pickle.load(f)
            return ball_detections

        ball_detections = self.infer_model(frames)

        if stub_path is not None:
            with open(stub_path, 'wb') as f:
                pickle.dump(ball_detections, f)

        return ball_detections

    def detect_frame(self, frame, prev_frame, preprev_frame):
        height, width = 360, 640
        img = cv2.resize(frame, (width, height))
        img_prev = cv2.resize(prev_frame, (width, height)) if prev_frame is not None else np.zeros_like(img)
        img_preprev = cv2.resize(preprev_frame, (width, height)) if preprev_frame is not None else np.zeros_like(img)
        imgs = np.concatenate((img, img_prev, img_preprev), axis=2)
        imgs = imgs.astype(np.float32) / 255.0
        imgs = np.rollaxis(imgs, 2, 0)
        inp = np.expand_dims(imgs, axis=0)
        with torch.no_grad():
            out = self.model(torch.from_numpy(inp).float().to(self.device))
        output = out.argmax(dim=1).detach().cpu().numpy()
        x_pred, y_pred = postprocess(output)
        # Convert (x, y) to pseudo-bounding box [x1, y1, x2, y2]
        if x_pred is not None and y_pred is not None:
            box_size = 10  # Increased box size for better compatibility
            ball_dict = {1: [x_pred - box_size, y_pred - box_size, x_pred + box_size, y_pred + box_size]}
        else:
            ball_dict = {1: [None, None, None, None]}
        return ball_dict

    def infer_model(self, frames):
        ball_detections = []
        height, width = 360, 640
        # Initialize with empty detections for first two frames
        ball_detections.extend([{1: [None, None, None, None]}] * 2)
        for num in tqdm(range(2, len(frames))):
            ball_dict = self.detect_frame(frames[num], frames[num-1], frames[num-2])
            ball_detections.append(ball_dict)
        return ball_detections

    def draw_bboxes(self, video_frames, ball_detections):
        output_video_frames = []
        for frame, ball_dict in zip(video_frames, ball_detections):
            frame = frame.copy()
            for track_id, coords in ball_dict.items():
                x1, y1, x2, y2 = coords
                if x1 is not None and y1 is not None:
                    # Draw a small rectangle for the ball
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
                    # Add text for ball ID
                    cv2.putText(frame, f"Ball ID: {track_id}", (int(x1) - 10, int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
            output_video_frames.append(frame)
        return output_video_frames

In [58]:
import pandas as pd
import cv2

# Analyze ball speed and save to CSV
def analyze_ball_speed():
    # Parameters
    input_video_path = "/kaggle/input/test-tennis-dataset/test_dataset/video8.mp4"
    ball_model_path = "/kaggle/input/tennis_best_model/pytorch/default/1/model_best.pth"
    court_model_path = "/kaggle/input/keypoint_model/pytorch/default/1/keypoints_model.pth"
    output_csv_path = "/kaggle/working/ball_speed_analysis.csv"

    # Read video
    video_frames, fps = read_video(input_video_path)

    # Initialize trackers and detectors
    ball_tracker = BallTracker(model_path=ball_model_path)
    court_line_detector = CourtLineDetector(court_model_path)
    mini_court = MiniCourt(video_frames[0])

    # Detect ball and court keypoints
    ball_detections = ball_tracker.detect_frames(video_frames)
    ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)
    court_keypoints = court_line_detector.predict(video_frames[0])

    # Debug: Inspect ball detections
    print(f"Sample ball_detections: {ball_detections[:5]}")

    # Convert ball positions to mini-court coordinates
    _, ball_mini_court_detections = mini_court.convert_bounding_boxes_to_mini_court_coordinates(
        {}, ball_detections, court_keypoints
    )

    # Debug: Inspect detections and shot frames
    print(f"Number of ball_mini_court_detections: {len(ball_mini_court_detections)}")
    print(f"Sample ball_mini_court_detections: {ball_mini_court_detections[:5]}")

    # Get ball shot frames
    ball_shot_frames = ball_tracker.get_ball_shot_frames(ball_detections)
    print(f"Number of shot frames: {len(ball_shot_frames)}")
    print(f"Shot frames: {ball_shot_frames}")

    # Check if ball_shot_frames is empty
    if len(ball_shot_frames) < 2:
        print("No sufficient ball shot frames detected for speed analysis.")
        return

    # Analyze ball speed
    ball_speed_data = []
    for ball_shot_ind in range(len(ball_shot_frames) - 1):
        start_frame = ball_shot_frames[ball_shot_ind]
        end_frame = ball_shot_frames[ball_shot_ind + 1]
        ball_shot_time_in_seconds = (end_frame - start_frame) / fps

        # Calculate distance covered by the ball
        try:
            start_coords = ball_mini_court_detections[start_frame][1]  # [x, y]
            end_coords = ball_mini_court_detections[end_frame][1]      # [x, y]
            if start_coords[0] is None or end_coords[0] is None:
                print(f"Skipping shot {ball_shot_ind + 1}: Missing coordinates")
                continue
            distance_covered_by_ball_pixels = measure_distance(start_coords, end_coords)
            distance_covered_by_ball_meters = convert_pixel_distance_to_meters(
                distance_covered_by_ball_pixels,
                constants.DOUBLE_LINE_WIDTH,
                mini_court.get_width_of_mini_court()
            )

            # Calculate speed in km/h
            speed_of_ball_shot = distance_covered_by_ball_meters / ball_shot_time_in_seconds * 3.6

            ball_speed_data.append({
                'shot_number': ball_shot_ind + 1,
                'start_frame': start_frame,
                'end_frame': end_frame,
                'distance_meters': distance_covered_by_ball_meters,
                'time_seconds': ball_shot_time_in_seconds,
                'speed_kmh': speed_of_ball_shot
            })
        except (KeyError, IndexError, TypeError) as e:
            print(f"Error processing shot {ball_shot_ind + 1}: {e}")
            continue

    # Save to CSV
    if ball_speed_data:
        ball_speed_df = pd.DataFrame(ball_speed_data)
        ball_speed_df.to_csv(output_csv_path, index=False)
        print(f"Ball speed analysis saved to {output_csv_path}")
    else:
        print("No valid ball speed data to save.")

# Run analysis
analyze_ball_speed()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 481/481 [00:23<00:00, 20.21it/s]


Sample ball_detections: [{1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}]
Number of ball_mini_court_detections: 483
Sample ball_mini_court_detections: [{1: (1122.764137088633, 99.64127186694358)}, {1: (1122.764137088633, 99.64127186694358)}, {1: (1122.764137088633, 99.64127186694358)}, {1: (1122.764137088633, 99.64127186694358)}, {1: (1122.764137088633, 99.64127186694358)}]
Number of shot frames: 9
Shot frames: [37, 120, 152, 189, 224, 261, 291, 324, 400]
Ball speed analysis saved to /kaggle/working/ball_speed_analysis.csv


In [71]:
import cv2
import numpy as np
import sys
sys.path.append('../')

class MiniCourtPlayerPosition():
    def __init__(self, frame):
        self.drawing_rectangle_width = 250
        self.drawing_rectangle_height = 500
        self.buffer = 50
        self.padding_court = 20

        self.set_canvas_background_box_position(frame)
        self.set_mini_court_position()
        self.set_court_drawing_key_points()
        self.set_court_lines()

    def convert_meters_to_pixels(self, meters):
        return convert_meters_to_pixel_distance(meters,
                                               constants.DOUBLE_LINE_WIDTH,
                                               self.court_drawing_width)

    def set_court_drawing_key_points(self):
        drawing_key_points = [0]*28

        # point 0 
        drawing_key_points[0], drawing_key_points[1] = int(self.court_start_x), int(self.court_start_y)
        # point 1
        drawing_key_points[2], drawing_key_points[3] = int(self.court_end_x), int(self.court_start_y)
        # point 2
        drawing_key_points[4] = int(self.court_start_x)
        drawing_key_points[5] = self.court_start_y + self.convert_meters_to_pixels(constants.HALF_COURT_LINE_HEIGHT*2)
        # point 3
        drawing_key_points[6] = drawing_key_points[0] + self.court_drawing_width
        drawing_key_points[7] = drawing_key_points[5] 
        # point 4
        drawing_key_points[8] = drawing_key_points[0] + self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[9] = drawing_key_points[1] 
        # point 5
        drawing_key_points[10] = drawing_key_points[4] + self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[11] = drawing_key_points[5] 
        # point 6
        drawing_key_points[12] = drawing_key_points[2] - self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[13] = drawing_key_points[3] 
        # point 7
        drawing_key_points[14] = drawing_key_points[6] - self.convert_meters_to_pixels(constants.DOUBLE_ALLY_DIFFERENCE)
        drawing_key_points[15] = drawing_key_points[7] 
        # point 8
        drawing_key_points[16] = drawing_key_points[8] 
        drawing_key_points[17] = drawing_key_points[9] + self.convert_meters_to_pixels(constants.NO_MANS_LAND_HEIGHT)
        # point 9
        drawing_key_points[18] = drawing_key_points[16] + self.convert_meters_to_pixels(constants.SINGLE_LINE_WIDTH)
        drawing_key_points[19] = drawing_key_points[17] 
        # point 10
        drawing_key_points[20] = drawing_key_points[10] 
        drawing_key_points[21] = self.court_start_y + self.convert_meters_to_pixels(constants.HALF_COURT_LINE_HEIGHT*2 - constants.NO_MANS_LAND_HEIGHT)
        # point 11
        drawing_key_points[22] = drawing_key_points[20] + self.convert_meters_to_pixels(constants.SINGLE_LINE_WIDTH)
        drawing_key_points[23] = drawing_key_points[21] 
        # point 12
        drawing_key_points[24] = int((drawing_key_points[16] + drawing_key_points[18])/2)
        drawing_key_points[25] = drawing_key_points[17] 
        # point 13
        drawing_key_points[26] = int((drawing_key_points[20] + drawing_key_points[22])/2)
        drawing_key_points[27] = drawing_key_points[21] 

        self.drawing_key_points = drawing_key_points

    def set_court_lines(self):
        self.lines = [
            (0, 2),
            (4, 5),
            (6, 7),
            (1, 3),
            (0, 1),
            (8, 9),
            (10, 11),
            (2, 3)
        ]

    def set_mini_court_position(self):
        self.court_start_x = self.start_x + self.padding_court
        self.court_start_y = self.start_y + self.padding_court
        self.court_end_x = self.end_x - self.padding_court
        self.court_end_y = self.end_y - self.padding_court
        self.court_drawing_width = self.court_end_x - self.court_start_x

    def set_canvas_background_box_position(self, frame):
        frame = frame.copy()
        self.end_x = frame.shape[1] - self.buffer
        self.end_y = self.buffer + self.drawing_rectangle_height
        self.start_x = self.end_x - self.drawing_rectangle_width
        self.start_y = self.end_y - self.drawing_rectangle_height

    def draw_court(self, frame):
        for i in range(0, len(self.drawing_key_points), 2):
            x = int(self.drawing_key_points[i])
            y = int(self.drawing_key_points[i+1])
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)

        for line in self.lines:
            start_point = (int(self.drawing_key_points[line[0]*2]), int(self.drawing_key_points[line[0]*2+1]))
            end_point = (int(self.drawing_key_points[line[1]*2]), int(self.drawing_key_points[line[1]*2+1]))
            cv2.line(frame, start_point, end_point, (0, 0, 0), 2)

        net_start_point = (self.drawing_key_points[0], int((self.drawing_key_points[1] + self.drawing_key_points[5])/2))
        net_end_point = (self.drawing_key_points[2], int((self.drawing_key_points[1] + self.drawing_key_points[5])/2))
        cv2.line(frame, net_start_point, net_end_point, (255, 0, 0), 2)

        return frame

    def draw_background_rectangle(self, frame):
        shapes = np.zeros_like(frame, np.uint8)
        cv2.rectangle(shapes, (self.start_x, self.start_y), (self.end_x, self.end_y), (255, 255, 255), cv2.FILLED)
        out = frame.copy()
        alpha = 0.5
        mask = shapes.astype(bool)
        out[mask] = cv2.addWeighted(frame, alpha, shapes, 1 - alpha, 0)[mask]
        return out

    def draw_mini_court(self, frames):
        output_frames = []
        for frame in frames:
            frame = self.draw_background_rectangle(frame)
            frame = self.draw_court(frame)
            output_frames.append(frame)
        return output_frames

    def get_start_point_of_mini_court(self):
        return (self.court_start_x, self.court_start_y)

    def get_width_of_mini_court(self):
        return self.court_drawing_width

    def get_court_drawing_keypoints(self):
        return self.drawing_key_points

    def get_mini_court_coordinates(self,
                                   object_position,
                                   closest_key_point, 
                                   closest_key_point_index, 
                                   reference_height_in_pixels,
                                   reference_height_in_meters):
        distance_from_keypoint_x_pixels, distance_from_keypoint_y_pixels = measure_xy_distance(object_position, closest_key_point)
        distance_from_keypoint_x_meters = convert_pixel_distance_to_meters(distance_from_keypoint_x_pixels,
                                                                         reference_height_in_meters,
                                                                         reference_height_in_pixels)
        distance_from_keypoint_y_meters = convert_pixel_distance_to_meters(distance_from_keypoint_y_pixels,
                                                                         reference_height_in_meters,
                                                                         reference_height_in_pixels)
        mini_court_x_distance_pixels = self.convert_meters_to_pixels(distance_from_keypoint_x_meters)
        mini_court_y_distance_pixels = self.convert_meters_to_pixels(distance_from_keypoint_y_meters)
        closest_mini_court_keypoint = (self.drawing_key_points[closest_key_point_index*2],
                                      self.drawing_key_points[closest_key_point_index*2+1])
        mini_court_position = (closest_mini_court_keypoint[0] + mini_court_x_distance_pixels,
                               closest_mini_court_keypoint[1] + mini_court_y_distance_pixels)
        return mini_court_position

    def convert_bounding_boxes_to_mini_court_coordinates(self, player_boxes, ball_boxes, original_court_key_points):
        output_player_boxes = []
        output_ball_boxes = []
        default_reference_height_meters = constants.PLAYER_1_HEIGHT_METERS
        default_reference_height_pixels = 100

        for frame_num, (player_bbox, ball_bbox) in enumerate(zip(player_boxes, ball_boxes)):
            print(f"Processing frame {frame_num}, player_bbox: {player_bbox}, ball_bbox: {ball_bbox}")
            output_player_bboxes_dict = {}

            # Process players
            for player_id, bbox in player_bbox.items():
                if not bbox or any(v is None for v in bbox):
                    print(f"Skipping player_id {player_id} in frame {frame_num}: invalid bbox {bbox}")
                    continue
                foot_position = get_foot_position(bbox)
                print(f"Player_id {player_id}, foot_position: {foot_position}")

                closest_key_point_index = get_closest_keypoint_index(foot_position, original_court_key_points, [0, 2, 12, 13])
                closest_key_point = (original_court_key_points[closest_key_point_index*2], 
                                    original_court_key_points[closest_key_point_index*2+1])
                print(f"Player_id {player_id}, closest keypoint index: {closest_key_point_index}, point: {closest_key_point}")

                try:
                    mini_court_player_position = self.get_mini_court_coordinates(
                        foot_position,
                        closest_key_point,
                        closest_key_point_index,
                        default_reference_height_pixels,
                        default_reference_height_meters
                    )
                    print(f"Player_id {player_id}, mini_court_position: {mini_court_player_position}")
                    output_player_bboxes_dict[player_id] = mini_court_player_position
                except Exception as e:
                    print(f"Error mapping player_id {player_id} in frame {frame_num}: {e}")
                    continue

            output_player_boxes.append(output_player_bboxes_dict)

            # Process ball
            ball_box = ball_bbox.get(1, [None, None, None, None])
            if ball_box[0] is None:
                output_ball_boxes.append({1: [None, None]})
                continue

            ball_position = get_center_of_bbox(ball_box)
            closest_key_point_index = get_closest_keypoint_index(ball_position, original_court_key_points, [0, 2, 12, 13])
            closest_key_point = (original_court_key_points[closest_key_point_index*2], 
                                original_court_key_points[closest_key_point_index*2+1])
            try:
                mini_court_ball_position = self.get_mini_court_coordinates(
                    ball_position,
                    closest_key_point,
                    closest_key_point_index,
                    default_reference_height_pixels,
                    default_reference_height_meters
                )
                print(f"Ball, frame {frame_num}, mini_court_position: {mini_court_ball_position}")
                output_ball_boxes.append({1: mini_court_ball_position})
            except Exception as e:
                print(f"Error mapping ball in frame {frame_num}: {e}")
                output_ball_boxes.append({1: [None, None]})

        print(f"Output player boxes length: {len(output_player_boxes)}, ball boxes length: {len(output_ball_boxes)}")
        return output_player_boxes, output_ball_boxes

    def convert_player_bounding_boxes_to_mini_court_coordinates(self, player_boxes, original_court_key_points):
        output_player_boxes = []
        default_reference_height_meters = constants.PLAYER_1_HEIGHT_METERS
        default_reference_height_pixels = 100

        for frame_num, player_bbox in enumerate(player_boxes):
            print(f"Processing frame {frame_num}, player_bbox: {player_bbox}")
            output_player_bboxes_dict = {}

            for player_id, bbox in player_bbox.items():
                if not bbox or any(v is None for v in bbox):
                    print(f"Skipping player_id {player_id} in frame {frame_num}: invalid bbox {bbox}")
                    continue
                foot_position = get_foot_position(bbox)
                print(f"Player_id {player_id}, foot_position: {foot_position}")

                closest_key_point_index = get_closest_keypoint_index(foot_position, original_court_key_points, [0, 2, 12, 13])
                closest_key_point = (original_court_key_points[closest_key_point_index*2], 
                                    original_court_key_points[closest_key_point_index*2+1])
                print(f"Closest keypoint index: {closest_key_point_index}, point: {closest_key_point}")

                try:
                    mini_court_player_position = self.get_mini_court_coordinates(
                        foot_position,
                        closest_key_point,
                        closest_key_point_index,
                        default_reference_height_pixels,
                        default_reference_height_meters
                    )
                    print(f"Player_id {player_id}, mini_court_position: {mini_court_player_position}")
                    output_player_bboxes_dict[player_id] = mini_court_player_position
                except Exception as e:
                    print(f"Error mapping player_id {player_id} in frame {frame_num}: {e}")
                    continue

            output_player_boxes.append(output_player_bboxes_dict)

        print(f"Output player boxes length: {len(output_player_boxes)}")
        return output_player_boxes

    def draw_points_on_mini_court(self, frames, positions, color=(0, 255, 0)):
        for frame_num, frame in enumerate(frames):
            if frame_num >= len(positions):
                continue
            for _, position in positions[frame_num].items():
                if position[0] is None:
                    continue
                x, y = position
                x = int(x)
                y = int(y)
                cv2.circle(frame, (x, y), 5, color, -1)
        return frames

In [72]:
from ultralytics import YOLO 
import cv2
import pickle
import sys
sys.path.append('../')

class PlayerTracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def choose_and_filter_players(self, court_keypoints, player_detections):
        player_detections_first_frame = player_detections[0]
        print(f"Player detections in first frame: {player_detections_first_frame}")
        chosen_players = self.choose_players(court_keypoints, player_detections_first_frame)
        print(f"Chosen players: {chosen_players}")
        if not chosen_players:
            print("Warning: No players chosen; returning empty detections")
            return [{} for _ in player_detections]
        filtered_player_detections = []
        for player_dict in player_detections:
            filtered_player_dict = {track_id: bbox for track_id, bbox in player_dict.items() if track_id in chosen_players}
            filtered_player_detections.append(filtered_player_dict)
        return filtered_player_detections

    def choose_players(self, court_keypoints, player_dict):
        distances = []
        for track_id, bbox in player_dict.items():
            player_center = get_center_of_bbox(bbox)
            min_distance = float('inf')
            for i in range(0, len(court_keypoints), 2):
                court_keypoint = (court_keypoints[i], court_keypoints[i+1])
                distance = measure_distance(player_center, court_keypoint)
                if distance < min_distance:
                    min_distance = distance
            distances.append((track_id, min_distance))
        
        # Sort distances in ascending order
        distances.sort(key=lambda x: x[1])
        # Choose up to 2 players, or fewer if not enough detected
        chosen_players = [dist[0] for dist in distances[:2]] if distances else []
        return chosen_players

    def detect_frames(self, frames, read_from_stub=False, stub_path=None):
        player_detections = []

        if read_from_stub and stub_path is not None:
            with open(stub_path, 'rb') as f:
                player_detections = pickle.load(f)
            return player_detections

        for frame in frames:
            player_dict = self.detect_frame(frame)
            player_detections.append(player_dict)
        
        if stub_path is not None:
            with open(stub_path, 'wb') as f:
                pickle.dump(player_detections, f)
        
        return player_detections

    def detect_frame(self, frame):
        # Lower confidence threshold to capture more detections
        results = self.model.track(frame, persist=True, conf=0.3)[0]
        id_name_dict = results.names
        print(f"YOLO class names: {id_name_dict}")
        print(f"Detected boxes: {len(results.boxes)}")
        player_dict = {}
        for box in results.boxes:
            if box.id is None or box.cls is None:
                continue
            track_id = int(box.id.tolist()[0])
            result = box.xyxy.tolist()[0]
            object_cls_id = box.cls.tolist()[0]
            object_cls_name = id_name_dict[object_cls_id]
            confidence = box.conf.tolist()[0] if box.conf is not None else 0.0
            print(f"Box: track_id={track_id}, class={object_cls_name}, confidence={confidence}, bbox={result}")
            # Allow "person" or similar classes (e.g., "player")
            if object_cls_name in ["person", "player"]:
                player_dict[track_id] = result
        
        return player_dict

    def draw_bboxes(self, video_frames, player_detections):
        output_video_frames = []
        for frame, player_dict in zip(video_frames, player_detections):
            frame = frame.copy()
            for track_id, bbox in player_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Player ID: {track_id}", (int(x1), int(y1 - 10)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            output_video_frames.append(frame)
        
        return output_video_frames

In [79]:
import pandas as pd
import cv2

def analyze_player_positions():
    # Parameters
    input_video_path = "/kaggle/input/test-tennis-dataset/test_dataset/video8.mp4"
    player_model_path = "/kaggle/input/player_detection_model/pytorch/default/1/best.pt"
    court_model_path = "/kaggle/input/keypoint_model/pytorch/default/1/keypoints_model.pth"
    output_csv_path = "/kaggle/working/player_positions_analysis.csv"

    # Read video
    video_frames, fps = read_video(input_video_path)
    print(f"Total frames: {len(video_frames)}, FPS: {fps}")

    # Initialize trackers and detectors
    player_tracker = PlayerTracker(model_path=player_model_path)
    court_line_detector = CourtLineDetector(court_model_path)
    mini_court = MiniCourtPlayerPosition(video_frames[0])

    # Detect players and court keypoints
    player_detections = player_tracker.detect_frames(video_frames)
    print(f"Sample player_detections (first 5 frames): {player_detections[:5]}")
    court_keypoints = court_line_detector.predict(video_frames[0])
    print(f"Court keypoints: {court_keypoints}")
    player_detections = player_tracker.choose_and_filter_players(court_keypoints, player_detections)

    # Convert player positions to mini-court coordinates
    player_mini_court_detections = mini_court.convert_player_bounding_boxes_to_mini_court_coordinates(
        player_detections, court_keypoints
    )
    print(f"Player mini-court detections length: {len(player_mini_court_detections)}")
    print(f"Sample player_mini_court_detections (first 5 frames): {player_mini_court_detections[:5]}")

    # Prepare data for CSV
    player_positions_data = []
    for frame_num, detections in enumerate(player_mini_court_detections):
        for player_id, position in detections.items():
            if position[0] is None or position[1] is None:
                continue
            player_positions_data.append({
                'frame_num': frame_num,
                'player_id': player_id,
                'x_position': position[0],
                'y_position': position[1]
            })

    # Save to CSV
    if player_positions_data:
        player_positions_df = pd.DataFrame(player_positions_data)
        player_positions_df.to_csv(output_csv_path, index=False)
        print(f"Player positions analysis saved to {output_csv_path}")
        print(f"Number of position records: {len(player_positions_data)}")
    else:
        print("No valid player position data to save.")

# Run analysis
analyze_player_positions()

Total frames: 483, FPS: 25


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



0: 384x640 2 players, 9.7ms
Speed: 1.8ms preprocess, 9.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
YOLO class names: {0: 'ball', 1: 'player'}
Detected boxes: 2
Box: track_id=1, class=player, confidence=0.8059882521629333, bbox=[783.1878051757812, 122.94021606445312, 833.2586059570312, 230.17001342773438]
Box: track_id=2, class=player, confidence=0.7898412942886353, bbox=[350.4493408203125, 492.1485595703125, 431.1181640625, 681.3565673828125]

0: 384x640 2 players, 9.4ms
Speed: 1.6ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
YOLO class names: {0: 'ball', 1: 'player'}
Detected boxes: 2
Box: track_id=1, class=player, confidence=0.8044515252113342, bbox=[782.9365234375, 122.54227447509766, 833.6766357421875, 230.07546997070312]
Box: track_id=2, class=player, confidence=0.7970708012580872, bbox=[350.01751708984375, 492.7787170410156, 433.7568054199219, 681.1234741210938]

0: 384x640 2 players, 8.0ms
Speed: 1.8ms preprocess,

In [74]:
import pandas as pd
import cv2

# Analyze opponent speed and save to CSV
def analyze_opponent_speed():
    # Parameters
    input_video_path = "/kaggle/input/test-tennis-dataset/test_dataset/video8.mp4"
    player_model_path = "/kaggle/input/player_detection_model/pytorch/default/1/best.pt"
    ball_model_path = "/kaggle/input/tennis_best_model/pytorch/default/1/model_best.pth"
    court_model_path = "/kaggle/input/keypoint_model/pytorch/default/1/keypoints_model.pth"
    output_csv_path = "/kaggle/working/opponent_speed_analysis.csv"

    # Read video
    video_frames, fps = read_video(input_video_path)
    print(f"Total frames: {len(video_frames)}, FPS: {fps}")

    # Initialize trackers and detectors
    player_tracker = PlayerTracker(model_path=player_model_path)
    ball_tracker = BallTracker(model_path=ball_model_path)
    court_line_detector = CourtLineDetector(court_model_path)
    mini_court = MiniCourtPlayerPosition(video_frames[0])

    # Detect players, ball, and court keypoints
    player_detections = player_tracker.detect_frames(video_frames)
    print(f"Sample player_detections (first 5 frames): {player_detections[:5]}")
    ball_detections = ball_tracker.detect_frames(video_frames)
    ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)
    print(f"Sample ball_detections (first 5 frames): {ball_detections[:5]}")
    court_keypoints = court_line_detector.predict(video_frames[0])
    print(f"Court keypoints: {court_keypoints}")
    player_detections = player_tracker.choose_and_filter_players(court_keypoints, player_detections)

    # Check if player detections are empty
    if not any(player_detections):
        print("No player detections found. Skipping analysis.")
        return

    # Convert positions to mini-court coordinates
    player_mini_court_detections, ball_mini_court_detections = mini_court.convert_bounding_boxes_to_mini_court_coordinates(
        player_detections, ball_detections, court_keypoints
    )
    print(f"Player mini-court detections length: {len(player_mini_court_detections)}")
    print(f"Sample player_mini_court_detections (first 5 frames): {player_mini_court_detections[:5]}")
    print(f"Ball mini-court detections length: {len(ball_mini_court_detections)}")
    print(f"Sample ball_mini_court_detections (first 5 frames): {ball_mini_court_detections[:5]}")

    # Get ball shot frames
    ball_shot_frames = ball_tracker.get_ball_shot_frames(ball_detections)
    print(f"Ball shot frames: {ball_shot_frames}")

    # Analyze opponent speed
    opponent_speed_data = []
    for ball_shot_ind in range(len(ball_shot_frames) - 1):
        start_frame = ball_shot_frames[ball_shot_ind]
        end_frame = ball_shot_frames[ball_shot_ind + 1]
        print(f"Processing shot {ball_shot_ind + 1}: start_frame={start_frame}, end_frame={end_frame}")

        # Check for valid player and ball positions
        if (start_frame >= len(player_mini_court_detections) or
            end_frame >= len(player_mini_court_detections) or
            not player_mini_court_detections[start_frame] or
            not player_mini_court_detections[end_frame] or
            not ball_mini_court_detections[start_frame].get(1) or
            ball_mini_court_detections[start_frame][1][0] is None):
            print(f"Skipping shot {ball_shot_ind + 1}: missing player or ball positions")
            continue

        player_positions = player_mini_court_detections[start_frame]
        ball_position = ball_mini_court_detections[start_frame][1]

        # Identify player who shot the ball
        try:
            player_shot_ball = min(
                player_positions.keys(),
                key=lambda player_id: measure_distance(
                    player_positions[player_id],
                    ball_position
                )
            )
        except ValueError as e:
            print(f"Skipping shot {ball_shot_ind + 1}: no player positions available ({e})")
            continue

        ball_shot_time_in_seconds = (end_frame - start_frame) / fps

        # Calculate opponent speed
        opponent_player_id = 1 if player_shot_ball == 2 else 2
        if opponent_player_id not in player_mini_court_detections[start_frame] or \
           opponent_player_id not in player_mini_court_detections[end_frame]:
            print(f"Skipping shot {ball_shot_ind + 1}: opponent {opponent_player_id} not detected")
            continue

        try:
            distance_covered_by_opponent_pixels = measure_distance(
                player_mini_court_detections[start_frame][opponent_player_id],
                player_mini_court_detections[end_frame][opponent_player_id]
            )
            distance_covered_by_opponent_meters = convert_pixel_distance_to_meters(
                distance_covered_by_opponent_pixels,
                constants.DOUBLE_LINE_WIDTH,
                mini_court.get_width_of_mini_court()
            )
            speed_of_opponent = distance_covered_by_opponent_meters / ball_shot_time_in_seconds * 3.6

            opponent_speed_data.append({
                'shot_number': ball_shot_ind + 1,
                'start_frame': start_frame,
                'end_frame': end_frame,
                'opponent_player_id': opponent_player_id,
                'distance_meters': distance_covered_by_opponent_meters,
                'time_seconds': ball_shot_time_in_seconds,
                'speed_kmh': speed_of_opponent
            })
        except Exception as e:
            print(f"Error computing speed for shot {ball_shot_ind + 1}: {e}")
            continue

    # Save to CSV
    if opponent_speed_data:
        opponent_speed_df = pd.DataFrame(opponent_speed_data)
        opponent_speed_df.to_csv(output_csv_path, index=False)
        print(f"Opponent speed analysis saved to {output_csv_path}")
        print(f"Number of speed records: {len(opponent_speed_data)}")
    else:
        print("No valid opponent speed data to save.")

# Run analysis
analyze_opponent_speed()

Total frames: 483, FPS: 25

0: 384x640 2 players, 9.4ms
Speed: 1.7ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
YOLO class names: {0: 'ball', 1: 'player'}
Detected boxes: 2
Box: track_id=1, class=player, confidence=0.8059882521629333, bbox=[783.1878051757812, 122.94021606445312, 833.2586059570312, 230.17001342773438]
Box: track_id=2, class=player, confidence=0.7898412942886353, bbox=[350.4493408203125, 492.1485595703125, 431.1181640625, 681.3565673828125]

0: 384x640 2 players, 8.0ms
Speed: 1.6ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
YOLO class names: {0: 'ball', 1: 'player'}
Detected boxes: 2
Box: track_id=1, class=player, confidence=0.8044515252113342, bbox=[782.9365234375, 122.54227447509766, 833.6766357421875, 230.07546997070312]
Box: track_id=2, class=player, confidence=0.7970708012580872, bbox=[350.01751708984375, 492.7787170410156, 433.7568054199219, 681.1234741210938]

0: 384x640 2 players, 7.9

100%|██████████| 481/481 [00:23<00:00, 20.39it/s]


Sample ball_detections (first 5 frames): [{1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}, {1: [767.0, 121.0, 787.0, 141.0]}]
Court keypoints: [     435.88      213.36      981.77      214.33       202.9      614.08      1209.9      618.16      504.15      213.38      328.73      614.52      913.33      214.22      1083.1      617.67      479.81      270.36       936.8      271.32      392.43      471.66      1020.9      474.07      707.87      270.86
      705.57      472.98]
Player detections in first frame: {1: [783.1878051757812, 122.94021606445312, 833.2586059570312, 230.17001342773438], 2: [350.4493408203125, 492.1485595703125, 431.1181640625, 681.3565673828125]}
Chosen players: [2, 1]
Processing frame 0, player_bbox: {1: [783.1878051757812, 122.94021606445312, 833.2586059570312, 230.17001342773438], 2: [350.4493408203125, 492.1485595703125, 431.1181640625, 681.3565673828125]}, ball_bbox: {

In [75]:
import pandas as pd
import matplotlib.pyplot as plt

# Visualize ball speed, player positions, and opponent speed
def visualize_analysis():
    # Load CSV files
    ball_speed_df = pd.read_csv("/kaggle/working/ball_speed_analysis.csv")
    player_positions_df = pd.read_csv("/kaggle/working/player_positions_analysis.csv")
    opponent_speed_df = pd.read_csv("/kaggle/working/opponent_speed_analysis.csv")

    # Plot ball speed
    plt.figure(figsize=(10, 6))
    plt.plot(ball_speed_df['shot_number'], ball_speed_df['speed_kmh'], marker='o')
    plt.title('Ball Speed per Shot')
    plt.xlabel('Shot Number')
    plt.ylabel('Speed (km/h)')
    plt.grid(True)
    plt.savefig('/kaggle/working/ball_speed_plot.png')
    plt.close()

    # Plot player positions
    plt.figure(figsize=(10, 6))
    for player_id in player_positions_df['player_id'].unique():
        player_data = player_positions_df[player_positions_df['player_id'] == player_id]
        plt.scatter(player_data['x_position'], player_data['y_position'], label=f'Player {player_id}', alpha=0.5)
    plt.title('Player Positions on Mini-Court')
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.legend()
    plt.grid(True)
    plt.savefig('/kaggle/working/player_positions_plot.png')
    plt.close()

    # Plot opponent speed
    plt.figure(figsize=(10, 6))
    plt.plot(opponent_speed_df['shot_number'], opponent_speed_df['speed_kmh'], marker='o', color='orange')
    plt.title('Opponent Speed per Shot')
    plt.xlabel('Shot Number')
    plt.ylabel('Speed (km/h)')
    plt.grid(True)
    plt.savefig('/kaggle/working/opponent_speed_plot.png')
    plt.close()

    print("Visualizations saved to /kaggle/working/")

# Run visualization
visualize_analysis()

Visualizations saved to /kaggle/working/


In [83]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

def calculate_player_speeds():
    # Load player positions
    player_positions_df = pd.read_csv('/kaggle/working/player_positions_analysis.csv')
    # Load ball speed to get shot frames
    ball_speed_df = pd.read_csv('/kaggle/working/ball_speed_analysis.csv')

    # Initialize list to store speed records
    speed_records = []

    # Process each shot
    for _, shot in ball_speed_df.iterrows():
        start_frame = int(shot['start_frame'])
        end_frame = int(shot['end_frame'])
        shot_number = int(shot['shot_number'])

        # Filter player positions for this shot
        shot_positions = player_positions_df[(player_positions_df['frame_num'] >= start_frame) & 
                                            (player_positions_df['frame_num'] <= end_frame)]

        # Process each player
        for player_id in shot_positions['player_id'].unique():
            player_data = shot_positions[shot_positions['player_id'] == player_id].sort_values('frame_num')
            if len(player_data) < 2:
                print(f'Skipping player {player_id} in shot {shot_number}: insufficient position data')
                continue

            # Calculate distances between consecutive frames
            distances = []
            times = []
            for i in range(1, len(player_data)):
                prev_row = player_data.iloc[i-1]
                curr_row = player_data.iloc[i]
                prev_pos = (prev_row['x_position'], prev_row['y_position'])
                curr_pos = (curr_row['x_position'], curr_row['y_position'])
                dist = distance.euclidean(prev_pos, curr_pos)
                distances.append(dist)
                # Assume 25 fps (adjust if different)
                time_diff = (curr_row['frame_num'] - prev_row['frame_num']) / 25.0
                times.append(time_diff)

            # Calculate average speed in pixels per second
            if distances and times:
                total_distance = sum(distances)
                total_time = sum(times)
                if total_time > 0:
                    speed_pixels_per_sec = total_distance / total_time
                    # Convert to km/h (assuming 1 pixel = 0.01 meters, adjust scaling as needed)
                    speed_m_per_sec = speed_pixels_per_sec * 0.01
                    speed_kmh = speed_m_per_sec * 3.6
                    speed_records.append({
                        'shot_number': shot_number,
                        'player_id': player_id,
                        'start_frame': start_frame,
                        'end_frame': end_frame,
                        'speed_kmh': speed_kmh
                    })
                else:
                    print(f'Skipping player {player_id} in shot {shot_number}: zero time difference')
            else:
                print(f'Skipping player {player_id} in shot {shot_number}: no valid distances')

    # Save to CSV
    speed_df = pd.DataFrame(speed_records)
    speed_df.to_csv('/kaggle/working/player_speed_analysis.csv', index=False)
    print(f'Player speed analysis saved to /kaggle/working/player_speed_analysis.csv')
    print(f'Number of speed records: {len(speed_df)}')

# Run the calculation
calculate_player_speeds()

Player speed analysis saved to /kaggle/working/player_speed_analysis.csv
Number of speed records: 14


In [85]:
import pandas as pd
import cv2
import numpy as np

def write_video(frames, output_path, fps):
    if not frames:
        print('No frames to write.')
        return

    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame in frames:
        out.write(frame)
    
    out.release()

def annotate_video_with_parameters():
    # Parameters
    input_video_path = '/kaggle/working/output_video_with_ball_and_players.avi'
    output_video_path = '/kaggle/working/annotated_video.mp4'
    player_speed_csv = '/kaggle/working/player_speed_analysis.csv'
    player_positions_csv = '/kaggle/working/player_positions_analysis.csv'
    ball_speed_csv = '/kaggle/working/ball_speed_analysis.csv'

    # Read CSV files
    player_speed_df = pd.read_csv(player_speed_csv)
    player_positions_df = pd.read_csv(player_positions_csv)
    ball_speed_df = pd.read_csv(ball_speed_csv)

    # Read video
    video_frames, fps = read_video(input_video_path)
    print(f'Total frames: {len(video_frames)}, FPS: {fps}')

    # Initialize mini-court
    mini_court = MiniCourtPlayerPosition(video_frames[0])

    # Prepare data for frame-wise lookup
    # Ball speed: map start_frame to speed_kmh
    ball_speed_dict = {}
    for _, row in ball_speed_df.iterrows():
        start_frame = int(row['start_frame'])
        end_frame = int(row['end_frame'])
        speed = row['speed_kmh']
        for frame in range(start_frame, end_frame + 1):
            ball_speed_dict[frame] = speed

    # Player speed: map frame_num to {player_id: speed_kmh}
    player_speed_dict = {}
    for _, row in player_speed_df.iterrows():
        start_frame = int(row['start_frame'])
        end_frame = int(row['end_frame'])
        speed = row['speed_kmh']
        player_id = int(row['player_id'])
        for frame in range(start_frame, end_frame + 1):
            if frame not in player_speed_dict:
                player_speed_dict[frame] = {}
            player_speed_dict[frame][player_id] = speed

    # Player positions: map frame_num to {player_id: (x, y)}
    player_positions_dict = {}
    for frame_num, group in player_positions_df.groupby('frame_num'):
        positions = {}
        for _, row in group.iterrows():
            player_id = int(row['player_id'])
            x = row['x_position']
            y = row['y_position']
            positions[player_id] = [x, y]
        player_positions_dict[int(frame_num)] = positions

    # Process frames
    output_frames = []
    for frame_num, frame in enumerate(video_frames):
        # Copy frame to avoid modifying the original
        frame = frame.copy()

        # Draw mini-court
        frame = mini_court.draw_background_rectangle(frame)
        frame = mini_court.draw_court(frame)

        # Draw player positions
        if frame_num in player_positions_dict:
            positions = player_positions_dict[frame_num]
            formatted_positions = [{pid: pos for pid, pos in positions.items()}]
            for player_id, position in positions.items():
                color = (0, 0, 255) if player_id == 1 else (255, 0, 0)
                frame_list = mini_court.draw_points_on_mini_court([frame], formatted_positions, color=color)
                frame = frame_list[0]
        else:
            print(f'No player positions for frame {frame_num}')

        # Overlay text for ball speed and player speeds
        text_y = frame.shape[0] - 100  # Start higher to fit all text
        text_x = frame.shape[1] - 300
        font_scale = 0.5  # Smaller font size for readability

        # Ball speed
        ball_speed_text = 'Ball Speed: N/A'
        if frame_num in ball_speed_dict:
            ball_speed_text = f'Ball Speed: {ball_speed_dict[frame_num]:.2f} km/h'
        cv2.putText(
            frame,
            ball_speed_text,
            (text_x, text_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            (255, 255, 255),
            1,
            cv2.LINE_AA
        )

        # Player speeds
        player1_speed_text = 'Player 1 Speed: N/A'
        player2_speed_text = 'Player 2 Speed: N/A'
        if frame_num in player_speed_dict:
            speeds = player_speed_dict[frame_num]
            if 1 in speeds:
                player1_speed_text = f'Player 1 Speed: {speeds[1]:.2f} km/h'
            if 2 in speeds:
                player2_speed_text = f'Player 2 Speed: {speeds[2]:.2f} km/h'
        cv2.putText(
            frame,
            player1_speed_text,
            (text_x, text_y + 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            (255, 255, 255),
            1,
            cv2.LINE_AA
        )
        cv2.putText(
            frame,
            player2_speed_text,
            (text_x, text_y + 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            (255, 255, 255),
            1,
            cv2.LINE_AA
        )

        output_frames.append(frame)
        print(f'Processed frame {frame_num}')

    # Write output video
    write_video(output_frames, output_video_path, fps)
    print(f'Annotated video saved to {output_video_path}')

# Run the annotation
annotate_video_with_parameters()

Total frames: 483, FPS: 25
Processed frame 0
Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed frame 50
Processed fram